In [1]:
print(f'lolmax')

lolmax


In [2]:
import sys
import argparse
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch as t
import importlib
import os
import pickle
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pandas import DataFrame, json_normalize
from tqdm import tqdm
import os
import numpy as np
from typing import Dict, Any, List, Tuple
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

In [3]:
sys.path.append('../linear-probes/')
from probes import LinearProbe
sys.path.append(os.path.dirname(os.path.abspath('.')))

from evaluate_utils import evaluate_probe_on_activation_dataset
from plots import plot_behavior_distribution, plot_roc_curves, add_roc_curves, print_metrics, plot_roc_curve_eval
import probes
from pprint import pprint as pp

from probe_datasets import AmongUsDataset, TruthfulQADataset, DishonestQADataset, RepEngDataset, RolePlayingDataset, ApolloProbeDataset
from configs import config_phi4, config_gpt2, config_llama3
base_config = config_phi4
amongus_expt_name: str = "2025-02-01_phi_phi_100_games_v3"
# layers_to_work_on: List[int] = list(range(base_config["num_layers"]))
layers_to_work_on: List[int] = [20]

In [4]:
from configs import config_phi4, config_gpt2, config_llama3
base_config = config_phi4

In [5]:
layer: int = layers_to_work_on[0]
probe_dataset_name: str = 'RepEngDataset'

probe = LinearProbe(base_config["activation_size"])
        
checkpoint_path = f'../linear-probes/checkpoints/{probe_dataset_name}_probe_{base_config["short_name"]}_layer{layer}.pkl'

with open(checkpoint_path, 'rb') as f:
    probe.model = pickle.load(f).model
    print(f'Loaded probe trained on {probe_dataset_name} for layer {layer}.')

config = base_config.copy()
config["layer"] = str(layer)

Loaded probe trained on RepEngDataset for layer 20.


In [6]:
dataset = AmongUsDataset(config, model=None, tokenizer=None, device='cpu', expt_name=config['expt_name'], test_split=1)
all_probe_outputs = []
chunk_size: int = 500
list_of_chunks_to_eval = [1]
row_indices = []
all_token_scores = []

In [7]:
for chunk_idx in tqdm(list_of_chunks_to_eval):
    test_acts_chunk = dataset.get_test_acts(chunk_idx)
    
    # Store the row indices for this chunk
    start_idx = chunk_idx * chunk_size
    end_idx = start_idx + len(test_acts_chunk)
    row_indices.extend(range(start_idx, end_idx))
    
    chunk_probe_outputs, _, token_scores = evaluate_probe_on_activation_dataset(
        chunk_data=test_acts_chunk,
        probe=probe,
        device=None,
        num_tokens=None,
        verbose=False,
        output_token_scores=True,
    )
    all_probe_outputs.extend(chunk_probe_outputs)
    all_token_scores.extend(token_scores)

100%|██████████| 1/1 [00:42<00:00, 42.36s/it]


In [17]:
from transformers import AutoTokenizer

model_name = config['model_name']
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
len(all_token_scores), len(chunk_probe_outputs)

(500, 500)

In [20]:
len(all_token_scores)

500

In [93]:
# full_prompt_tokens

In [50]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML

In [88]:
dataset.agent_logs_df.iloc[row_indices[33]]

game_index                                                                         Game 59
step                                                                                     1
timestamp                                                       2025-02-01 04:56:28.893432
player.name                                                                 Player 4: cyan
player.identity                                                                   Impostor
player.personality                                                                    None
player.model                                                               microsoft/phi-4
player.location                                                                  Cafeteria
interaction.system_prompt                \nYou are Player 4: cyan. You are an Impostor ...
interaction.prompt.Summarization                         No thought process has been made.
interaction.prompt.All Info              Game Time: 1/50\nCurrent phase: Meeting phase ...

In [101]:
def colorize(words, color_array):
    import plotly.colors as pc    
    colorscale = [[0, 'rgb(255,0,0)'], [1, 'rgb(255,255,255)']]
    template = '<span class="barcode"; style="color: black; background-color: {}; font-size: 14px">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        # Handle escape characters
        if word == '\n':
            colored_string += '<br>'
            continue
        elif word == '\n\n':
            colored_string += '<br><br>'
            continue
        elif word == '\n\n\n':
            colored_string += '<br><br><br>'
            continue
        elif word == '.\n':
            colored_string += '.<br>'
            continue
        elif word == '\t':
            colored_string += '&nbsp;&nbsp;&nbsp;&nbsp;'
            continue
        elif word == '\\' or word == '\r':
            # Skip other escape characters or handle them differently if needed
            continue
            
        rgb_color = pc.sample_colorscale(colorscale, color)[0]
        r, g, b = map(int, rgb_color.strip('rgb()').split(','))
        hex_color = f'#{r:02x}{g:02x}{b:02x}'
        colored_string += template.format(hex_color, '&nbsp' + word + '&nbsp')
    
    return colored_string

def visualize_token_scores(row_index: int = 42):
    row_token_scores = all_token_scores[row_index]
    row_token_scores = np.array(row_token_scores)
    raw_df = dataset.agent_logs_df
    actual_row_idx = row_indices[row_index]

    fp = dataset.agent_logs_row_to_full_prompt(raw_df.iloc[actual_row_idx])
    encoded = tokenizer.encode(fp, return_tensors="pt")

    full_prompt_tokens = []

    for token_id in encoded[0]:
        token = tokenizer.decode(token_id)
        full_prompt_tokens.append(token)

    words = full_prompt_tokens
    color_array = row_token_scores

    print(len(color_array))
    return colorize(words, color_array), all_token_scores[row_index]

In [110]:
def plot_running_average_scores(scores1, scores2, window_size=None):
    import plotly.graph_objects as go
    import numpy as np
    
    # Calculate running averages
    if window_size is None:
        # Default behavior: cumulative running average
        running_avg1 = np.cumsum(scores1) / np.arange(1, len(scores1) + 1)
        running_avg2 = np.cumsum(scores2) / np.arange(1, len(scores2) + 1)
    else:
        # Windowed running average
        running_avg1 = np.zeros(len(scores1))
        running_avg2 = np.zeros(len(scores2))
        
        for i in range(len(scores1)):
            start_idx = max(0, i + 1 - window_size)
            window_len = i + 1 - start_idx
            running_avg1[i] = np.sum(scores1[start_idx:i+1]) / window_len
            
        for i in range(len(scores2)):
            start_idx = max(0, i + 1 - window_size)
            window_len = i + 1 - start_idx
            running_avg2[i] = np.sum(scores2[start_idx:i+1]) / window_len
    
    # Create x-axis values (token indices)
    x_values = list(range(1, len(scores1) + 1))
    
    # Create figure
    fig = go.Figure()
    
    # Add traces for both running averages
    fig.add_trace(go.Scatter(
        x=x_values,
        y=running_avg1,
        mode='lines',
        name='Crewmate',
        line=dict(color='blue')
    ))
    
    fig.add_trace(go.Scatter(
        x=x_values,
        y=running_avg2,
        mode='lines',
        name='Impostor',
        line=dict(color='red')
    ))
    
    # Update layout
    fig.update_layout(
        title='',
        xaxis_title='Token Index',
        yaxis_title='Deception Probe Score (running average)',
        legend_title='Player',
        template='plotly_white',
        legend=dict(
            x=0.72,
            y=0.98,
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='rgba(0, 0, 0, 0.3)',
            borderwidth=1
        )
    )
    
    fig.update_layout(
        width=1000,
        height=600
    )
    
    return fig

In [108]:
# def colorize(words, color_array):
#     import plotly.colors as pc    
#     colorscale = [[0, 'rgb(255,0,0)'], [1, 'rgb(255,255,255)']]
#     template = '<span class="barcode"; style="color: black; background-color: {}; font-size: 14px">{}</span>'
#     colored_string = ''
#     for word, color in zip(words, color_array):
#         rgb_color = pc.sample_colorscale(colorscale, color)[0]
#         r, g, b = map(int, rgb_color.strip('rgb()').split(','))
#         hex_color = f'#{r:02x}{g:02x}{b:02x}'
#         colored_string += template.format(hex_color, '&nbsp' + word + '&nbsp')
    
#     return colored_string

# def visualize_token_scores(row_index: int = 42):
#     row_token_scores = all_token_scores[row_index]
#     row_token_scores = np.array(row_token_scores)
#     raw_df = dataset.agent_logs_df
#     actual_row_idx = row_indices[row_index]

#     fp = dataset.agent_logs_row_to_full_prompt(raw_df.iloc[actual_row_idx])
#     encoded = tokenizer.encode(fp, return_tensors="pt")

#     full_prompt_tokens = []

#     for token_id in encoded[0]:
#         token = tokenizer.decode(token_id)
#         full_prompt_tokens.append(token)

#     words = full_prompt_tokens
#     color_array = row_token_scores

#     print(len(color_array))
#     return colorize(words, color_array), raw_token_scores

In [102]:
HTML(visualize_token_scores(42)[0])

1189


In [103]:
HTML(visualize_token_scores(33)[0])

1254


In [115]:
list1 = visualize_token_scores(42)[1]
list2 = visualize_token_scores(33)[1]

# Plot the running averages
running_avg_plot = plot_running_average_scores(list1, list2, window_size=10000)
running_avg_plot.show()

1189
1254


In [116]:
list1 = visualize_token_scores(42)[1]
list2 = visualize_token_scores(33)[1]

# Plot the running averages
running_avg_plot = plot_running_average_scores(list1, list2, window_size=100)
running_avg_plot.show()

1189
1254
